In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

c:\Users\jaesc2\GitHub\skforecast
0.19.0


In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from skforecast.datasets import fetch_dataset
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

In [3]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "date"], "header": 0}
)
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.asfreq('MS')

╭────────────────────────────────────── h2o ───────────────────────────────────────╮
│ Description:                                                                     │
│ Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health    │
│ system had between 1991 and 2008.                                                │
│                                                                                  │
│ Source:                                                                          │
│ Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd        │
│ Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman │
│ /fpp3package, http://OTexts.com/fpp3.                                            │
│                                                                                  │
│ URL:                                                                             │
│ https://raw.githubusercontent.com/skforecast/skforecast-                         │
│ datasets/main/data/h2o.csv                                                       │
│                                                                                  │
│ Shape: 204 rows x 2 columns                                                      │
╰──────────────────────────────────────────────────────────────────────────────────╯

In [4]:
# Load model
# ==============================================================================
forecaster_loaded = load_forecaster(r'C:\Users\jaesc2\GitHub\skforecast\docs\user_guides\forecaster_001.joblib', verbose=True)

╭────────────────────────────── SkforecastVersionWarning ──────────────────────────────╮
│ The skforecast version installed in the environment differs from the version used to │
│ create the forecaster.                                                               │
│     Installed Version  : 0.19.0                                                      │
│     Forecaster Version : 0.18.0                                                      │
│ This may create incompatibilities when using the library.                            │
│                                                                                      │
│ Category : skforecast.exceptions.SkforecastVersionWarning                            │
│ Location : c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:2040          │
│ Suppress : warnings.simplefilter('ignore', category=SkforecastVersionWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

ForecasterRecursive 
Estimator: RandomForestRegressor 
Lags: [1 2 3 4 5] 
Window features: None 
Window size: 5 
Series name: y 
Exogenous included: False 
Exogenous names: None 
Transformer for y: None 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1991-07-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Estimator parameters: 
    {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth':
    None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None,
    'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2,
    'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100,
    'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0,
    'warm_start': False} 
fit_kwargs: {} 
Creation date: 2025-12-05 10:22:51 
Last fit date: 2025-12-05 10:22:51 
Skforecast version:

In [5]:
forecaster_loaded.predict(steps=12)

2008-07-01    0.714526
2008-08-01    0.789144
2008-09-01    0.818433
2008-10-01    0.845027
2008-11-01    0.914621
2008-12-01    0.965855
2009-01-01    1.001594
2009-02-01    1.046592
2009-03-01    1.002871
2009-04-01    0.967201
2009-05-01    0.778187
2009-06-01    0.653026
Freq: MS, Name: pred, dtype: float64